<a href="https://colab.research.google.com/github/Patric-Ramz/bdt-2023-26720051/blob/main/Post_Block_Assignment_3_Q4.3/BDT_PBA3_Q4_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=12c8402455a66e8d6b6f345d1c469155ebda9d4f17a7bbb61b2043a5f86f08eb
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("RandomForestExample").getOrCreate()

In [3]:
! wget https://storage.googleapis.com/bdt-spark-store/external_sources.csv -O gcs_external_sources.csv
! wget https://storage.googleapis.com/bdt-spark-store/internal_data.csv -O gcs_internal_data.csv

--2023-11-05 10:53:11--  https://storage.googleapis.com/bdt-spark-store/external_sources.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.193.207, 172.217.204.207, 172.217.203.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.193.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15503836 (15M) [text/csv]
Saving to: ‘gcs_external_sources.csv’

gcs_external_source 100%[===================>]  14.79M  15.2MB/s    in 1.0s    

2023-11-05 10:53:12 (15.2 MB/s) - ‘gcs_external_sources.csv’ saved [15503836/15503836]

--2023-11-05 10:53:13--  https://storage.googleapis.com/bdt-spark-store/internal_data.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.193.207, 172.217.204.207, 172.217.203.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.193.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 152978396 (146M) [text/csv]
Saving to: ‘gcs_int

In [4]:
import pyspark.pandas as pd

# Read in data from CSV files using the pandas API on Spark
pdf_data = pd.read_csv('gcs_internal_data.csv')
pdf_ext = pd.read_csv('gcs_external_sources.csv')

# Join them on their common identifier key
pdf_full = pdf_data.merge(pdf_ext, on='SK_ID_CURR')

# Select columns
columns_extract = ['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3',
                   'DAYS_BIRTH', 'DAYS_EMPLOYED', 'NAME_EDUCATION_TYPE',
                   'DAYS_ID_PUBLISH', 'CODE_GENDER', 'AMT_ANNUITY',
                   'DAYS_REGISTRATION', 'AMT_GOODS_PRICE', 'AMT_CREDIT',
                   'ORGANIZATION_TYPE', 'DAYS_LAST_PHONE_CHANGE',
                   'NAME_INCOME_TYPE', 'AMT_INCOME_TOTAL', 'OWN_CAR_AGE', 'TARGET']
pdf = pdf_full[columns_extract]

/usr/local/lib/python3.10/dist-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_csv`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [5]:
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder, Imputer, StandardScaler
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

df = pdf.to_spark()

# Split the data into train and test sets
(train, test) = df.randomSplit([0.8, 0.2], seed=101)

# Handle categorical variables with StringIndexer and OneHotEncoder
indexer = StringIndexer(inputCol="NAME_EDUCATION_TYPE", outputCol="NAME_EDUCATION_TYPE_Index")
encoder = OneHotEncoder(inputCols=["NAME_EDUCATION_TYPE_Index"], outputCols=["NAME_EDUCATION_TYPE_Vec"])

numeric_input_cols = [
    'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3',
    'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION',
    'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'AMT_CREDIT',
    'DAYS_LAST_PHONE_CHANGE', 'AMT_INCOME_TOTAL', 'OWN_CAR_AGE'
]

# Fill in missing data with Imputer
imputer = Imputer(inputCols=numeric_input_cols, outputCols=numeric_input_cols, strategy="median")

# Initialize the VectorAssembler
assembled_features = VectorAssembler(inputCols=numeric_input_cols, outputCol="features")

# Scale features with StandardScaler
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")

# Define the Random Forest model
rf = RandomForestClassifier(featuresCol="scaledFeatures", labelCol="TARGET", numTrees=100, seed=50)

# Chain indexers and the forest in a Pipeline
pipeline = Pipeline(stages=[indexer, encoder, imputer, assembled_features, scaler, rf])

# Train the model
model = pipeline.fit(train)

# Make predictions
predictions = model.transform(test)

# Select example rows to display
predictions.select("prediction", "TARGET").show(5)

# Select (prediction, true label) and compute test error
evaluator = BinaryClassificationEvaluator(labelCol="TARGET")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

/usr/local/lib/python3.10/dist-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


+----------+------+
|prediction|TARGET|
+----------+------+
|       0.0|     0|
|       0.0|     0|
|       0.0|     0|
|       0.0|     0|
|       0.0|     0|
+----------+------+
only showing top 5 rows

Test Error = 0.293573
